# Handling Large Volumes
This example shows how `robotools` automagically generates worklists for pipetting large volumes.

In [1]:
%load_ext autoreload
%autoreload 2

import numpy

from robotools import liquidhandling
from robotools import evotools

## Defining labwares
For a dilution series we start with different glucose concentrations in column 1 and dilute everything 1:1 over columns 1-4.

In [19]:
# we have troughs for water, glucose and 2x media
water = liquidhandling.Labware('Water', 1, 1,
                               min_volume=5000, max_volume=100*1000,
                               initial_volumes=100*1000, virtual_rows=8)
glucose = liquidhandling.Labware('Glucose', 1, 1,
                               min_volume=1000, max_volume=25*1000,
                               initial_volumes=20*1000, virtual_rows=8)

# an empty 48 Well DWP for preparing triplicate cultivations
dwp = liquidhandling.Labware('48DWP', 6, 8, min_volume=100, max_volume=4000)

print(water)
print(glucose)
print(dwp)

Water
[[100000.]]
Glucose
[[20000.]]
48DWP
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


## Pipetting with large volumes
In every pipetting step, the robot can handle a max of 950 µl.

When writing the worklist however, you can set `auto_split=True` (default) and the steps will be splitted by source column automatically.

In [20]:
with evotools.Worklist(max_volume=950, auto_split=True) as wl:
    wl.transfer(
        glucose, glucose.wells[:6,0],
        dwp, dwp.wells[:,0],
        [0,500,1000,1500,2500,2750],
        label='initial glucose'
    )
    wl.transfer(
        water, water.wells[:6,0],
        dwp, dwp.wells[:,0],
        [3000,2500,2000,1500,500,250],
        label='water up to 3 ml'
    )
    wl.transfer(
        water, water.wells[:6,0],
        dwp, dwp.wells[:,1],
        1500,
        label='water for first dilution'
    )
    wl.transfer(
        dwp, dwp.wells[:,0],
        dwp, dwp.wells[:,1],
        1500,
        label='2x dilution from col1 into col2'
    )

Our three labwares have been modified by the pipetting steps. We can inspect the history of the 96-well plate to check if we selected the right wells & volumes:

In [21]:
print(dwp.report)

48DWP
initial
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]

initial glucose
[[   0.    0.    0.    0.    0.    0.    0.    0.]
 [ 500.    0.    0.    0.    0.    0.    0.    0.]
 [1000.    0.    0.    0.    0.    0.    0.    0.]
 [1500.    0.    0.    0.    0.    0.    0.    0.]
 [2500.    0.    0.    0.    0.    0.    0.    0.]
 [2750.    0.    0.    0.    0.    0.    0.    0.]]

water up to 3 ml
[[3000.    0.    0.    0.    0.    0.    0.    0.]
 [3000.    0.    0.    0.    0.    0.    0.    0.]
 [3000.    0.    0.    0.    0.    0.    0.    0.]
 [3000.    0.    0.    0.    0.    0.    0.    0.]
 [3000.    0.    0.    0.    0.    0.    0.    0.]
 [3000.    0.    0.    0.    0.    0.    0.    0.]]

water for first dilution
[[3000. 1500.    0.    0.    0.    0.    0.    0.]
 [3000. 1500.    0.    0.    0.    0.    0.    0.]
 [3000. 1500.    0.    0.    0.    0.    0.   

## Inspecting the result
Relevant for the robot is just the worklist, which is merely a list of all individual aspirate/dispense/wash/... steps.

Note that it includes the `label`s as comments.

In [22]:
wl

C;initial glucose
A;Glucose;;;2;;500.00;;;;
D;48DWP;;;2;;500.00;;;;
W1;
A;Glucose;;;3;;500.00;;;;
D;48DWP;;;3;;500.00;;;;
W1;
A;Glucose;;;4;;750.00;;;;
D;48DWP;;;4;;750.00;;;;
W1;
A;Glucose;;;5;;834.00;;;;
D;48DWP;;;5;;834.00;;;;
W1;
A;Glucose;;;6;;917.00;;;;
D;48DWP;;;6;;917.00;;;;
W1;
B;
A;Glucose;;;3;;500.00;;;;
D;48DWP;;;3;;500.00;;;;
W1;
A;Glucose;;;4;;750.00;;;;
D;48DWP;;;4;;750.00;;;;
W1;
A;Glucose;;;5;;834.00;;;;
D;48DWP;;;5;;834.00;;;;
W1;
A;Glucose;;;6;;917.00;;;;
D;48DWP;;;6;;917.00;;;;
W1;
B;
A;Glucose;;;5;;832.00;;;;
D;48DWP;;;5;;832.00;;;;
W1;
A;Glucose;;;6;;916.00;;;;
D;48DWP;;;6;;916.00;;;;
W1;
B;
C;water up to 3 ml
A;Water;;;1;;750.00;;;;
D;48DWP;;;1;;750.00;;;;
W1;
A;Water;;;2;;834.00;;;;
D;48DWP;;;2;;834.00;;;;
W1;
A;Water;;;3;;667.00;;;;
D;48DWP;;;3;;667.00;;;;
W1;
A;Water;;;4;;750.00;;;;
D;48DWP;;;4;;750.00;;;;
W1;
A;Water;;;5;;500.00;;;;
D;48DWP;;;5;;500.00;;;;
W1;
A;Water;;;6;;250.00;;;;
D;48DWP;;;6;;250.00;;;;
W1;
B;
A;Water;;;1;;750.00;;;;
D;48DWP;;;1;;750.00;;